# MARA Graph

In [30]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import matplotlib.colors as mcolors
from matplotlib.cm import Blues

import matplotlib.pyplot as plt
import networkx as nx
from matplotlib.lines import Line2D 

In [31]:
def parse_strategies_from_log(log_file):
    """Extract defense strategies and attack paths from the log file"""
    defense_strategy = {}
    attack_paths = []
    attack_probs = []
    attack_rate = 0
    defense_rate = 0
    attacker_success = 0
    
    try:
        with open(log_file, 'r') as f:
            log_content = f.read()
            
        # Parse attack paths
        attack_paths_section = log_content.split('Debug - Current Attack paths:')[1].split('++++++')[0].strip()
        for line in attack_paths_section.strip().split('\n'):
            if ':' in line:
                path_index, path_str = line.strip().split(':', 1)
                attack_paths.append(path_str.strip())
        
        # Parse attack and defense rates
        if 'attack rate =' in log_content:
            rate_line = log_content.split('attack rate =')[1].split('\n')[0]
            attack_rate = int(rate_line.split(',')[0].strip())
            defense_rate = int(rate_line.split('defense rate =')[1].strip())
        
        # Parse defense strategy
        defense_section = log_content.split('optimal defense strategy:')[1].split('worst case attack strategies')[0]
        for line in defense_section.strip().split('\n')[1:]:  # Skip header
            if line.strip():
                parts = line.strip().split()
                if len(parts) >= 2:
                    node_id = int(parts[0])
                    probability = float(parts[1])
                    defense_strategy[node_id] = probability
        
        # Parse attack strategy probabilities
        attack_section = log_content.split('worst case attack strategies per goal:')[1].split('[1]')[0]
        for line in attack_section.strip().split('\n')[1:]:  # Skip header
            if line.strip():
                parts = line.strip().split()
                if len(parts) >= 2:
                    prob = float(parts[1])
                    attack_probs.append(prob)
        
        # Parse attacker success probability
        if 'Attacker can guarantee success probability of:' in log_content:
            success_line = log_content.split('Attacker can guarantee success probability of:')[1].strip()
            attacker_success = float(success_line.split()[0])
                    
        return defense_strategy, attack_paths, attack_probs, attack_rate, defense_rate, attacker_success
    
    except Exception as e:
        print(f"Error parsing log file: {e}")
        return {}, [], [], 0, 0, 0

In [32]:
def defense_color_mapping(probability, min_prob=0.05, max_prob=1.0):
    """Map defense probability to a blue gradient color"""
    # Normalize the probability to a value between 0 and 1
    if max_prob == min_prob:
        normalized = 1.0  # Avoid division by zero
    else:
        normalized = (probability - min_prob) / (max_prob - min_prob)
    
    # CHANGE THIS LINE:
    # From: color_value = 0.3 + (normalized * 0.6)
    # To: color_value = 0.5 + (normalized * 0.4)
    color_value = 0.65 + (normalized * 0.4)
    return Blues(color_value)

In [ ]:
def graph_plotting(basename):
    """Enhanced graph plotting with strategy information from log file"""
    basename = basename

    # Parse strategies from log file
    defense_strategy, attack_paths, attack_probs, attack_rate, defense_rate, attacker_success = parse_strategies_from_log(f"{basename}.log")

    # Filter defense strategies with probabilities < 5%
    filtered_defense = {node: prob for node, prob in defense_strategy.items() if prob >= 0.05}

    # Find min and max defense probabilities for gradient scaling
    if filtered_defense:
        min_prob = min(filtered_defense.values())
        max_prob = max(filtered_defense.values())
    else:
        min_prob, max_prob = 0.05, 1.0

    # Create a basic spring layout as starting point
    pos = nx.spring_layout(attack_graph, k=1, iterations=50, seed=42)

    # Identify key nodes (using actual node numbers in the graph)
    initial_node = 1  # Node 1 (green, entry node)
    target_nodes = [6, 9]  # Nodes 6 and 9 (red, target nodes)

    # Identify intermediate nodes
    all_nodes = list(attack_graph.nodes())
    intermediate_nodes = [node for node in all_nodes if node != initial_node and node not in target_nodes]

    # Calculate node distances from initial node (shortest path length)
    distances = {}
    for node in all_nodes:
        try:
            distances[node] = nx.shortest_path_length(attack_graph, initial_node, node)
        except nx.NetworkXNoPath:
            # No path from initial to this node, set a large distance
            distances[node] = len(attack_graph)

    # Normalize distances to determine y-coordinate
    max_dist = max(distances.values()) if distances else 1

    # Set y-coordinate based on distance from initial node
    for node in all_nodes:
        dist = distances.get(node, 0)
        pos[node][1] = 1.0 - (dist / max_dist if max_dist > 0 else 0)

    # Ensure initial node is at the top and target nodes at the bottom
    pos[initial_node][1] = 1.0  # Place at the top
    for node in target_nodes:
        if node in pos:
            pos[node][1] = 0.0 

    # Custom positioning adjustments
    pos[initial_node][0] = -0.2
    pos[initial_node][1] = 1.2
    pos[4][0] = -0.4
    pos[3][0] = -0.3
    pos[6][0] = -0.35
    pos[2][0] = -0.2
    pos[5][0] = 0
    pos[7][0] = -0.1
    pos[9][0] = -0.1  # Move Node 9 a lot to the right
    pos[8][0] = 0  # Move Node 8 moderately to the right

    # Create figure with GridSpec for better control of layout
    fig = plt.figure(figsize=(10, 8))
    gs = fig.add_gridspec(2, 1, height_ratios=[3, 1])  # 2 rows: graph and legends
    
    # Main network plot area
    ax_graph = fig.add_subplot(gs[0])
    
    # Draw the network in the main area
    nx.draw_networkx_edges(attack_graph, pos, ax=ax_graph, edge_color='gray', arrows=True, arrowsize=20, width=2)

    node_colors = []
    for node in attack_graph.nodes():
        if node == 1:
            node_colors.append('lightgreen')
        elif node in [6, 9]:
            node_colors.append('lightcoral')
        else:
            node_colors.append('lightblue')

    white_text_nodes = []
    node_list = list(attack_graph.nodes())
    for node, prob in filtered_defense.items():
        if node in node_list:
            idx = node_list.index(node)
            node_colors[idx] = defense_color_mapping(prob, min_prob, max_prob)
            if prob > (min_prob + max_prob) / 2:
                white_text_nodes.append(node)

    nx.draw_networkx_nodes(attack_graph, pos, ax=ax_graph, node_color=node_colors, 
                         node_size=700, edgecolors='black', linewidths=2)

    node_labels = {node: str(node) for node in attack_graph.nodes()}
    nx.draw_networkx_labels(attack_graph, pos, ax=ax_graph, labels=node_labels, 
                          font_size=12, font_weight='bold', font_color='black')
    white_labels = {node: str(node) for node in white_text_nodes if node in node_labels}
    if white_labels:
        nx.draw_networkx_labels(attack_graph, pos, ax=ax_graph, labels=white_labels, 
                              font_size=12, font_weight='bold', font_color='white')

    # Turn off axis for graph
    ax_graph.axis('off')


    if legends:
        # Add title to the figure
        ax_graph.set_title("Optimal Strategies (≥5%)", fontsize=12, fontweight='bold', pad=10)
        
        # Create a dedicated area for legends
        ax_legends = fig.add_subplot(gs[1])
        ax_legends.axis('off')  # Hide axes for legend area
        
        # Column 1: Stats (Fixed 3 Elements)
        col1_elements = [
            plt.Line2D([0],[0], linestyle="None", marker="", label=f'Attack rate: {attack_rate}'),
            plt.Line2D([0],[0], linestyle="None", marker="", label=f'Defense rate: {defense_rate}'),
            plt.Line2D([0],[0], linestyle="None", marker="", label=f'Attacker success: {attacker_success}')
        ]

        # Column 2: Defense nodes (Variable Elements)
        sorted_defense = sorted(filtered_defense.items(), key=lambda x: x[1], reverse=True)
        col2_elements = [
            plt.Line2D([0],[0], linestyle="None", marker='o', markerfacecolor=defense_color_mapping(prob, min_prob, max_prob),
                    markersize=10, label=f'Defense: Node {node} ({prob:.2f})') 
            for node, prob in sorted_defense
        ]

        # Column 3: Attack paths (Variable Elements, ≥5% probability)
        col3_elements = [
            plt.Line2D([0],[0], linestyle="-", color='black',
                    label=f'Attack path {i}: {path} ({prob:.2f})') 
            for i, (path, prob) in enumerate(zip(attack_paths, attack_probs)) if prob >= 0.05
        ]
        
        # Create three separate legends in the legend area with adjusted positions
        legend1 = ax_legends.legend(col1_elements, [elem.get_label() for elem in col1_elements], 
                                title="Stats",
                                loc='upper left', 
                                bbox_to_anchor=(-0.05, 1.0),  # Moved slightly left
                                frameon=False)
        ax_legends.add_artist(legend1)

        legend2 = ax_legends.legend(col2_elements, [elem.get_label() for elem in col2_elements], 
                                title="Defense Node Weights",
                                loc='upper center', 
                                bbox_to_anchor=(0.4, 1.0),  # Moved to the left
                                frameon=False)
        ax_legends.add_artist(legend2)

        legend3 = ax_legends.legend(col3_elements, [elem.get_label() for elem in col3_elements], 
                                title="Attack Path Weights",
                                loc='upper right', 
                                bbox_to_anchor=(1., 1.0),
                                frameon=False)
    
    # Ensure proper spacing
    plt.tight_layout()
    plt.show()

In [34]:
graph_plotting(basename)

NameError: name 'basename' is not defined